In [5]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('../database/pseudo_observations/vas.txt')
data1 = np.array(data1.iloc[::2, :])
data1 = np.array([float(i[0]) for i in data1])

data2 = pd.read_csv('../database/pseudo_observations/LQD.txt')
data2 = np.array(data2.iloc[::2, :])
data2 = np.array([float(i[0]) for i in data2])

data3 = pd.read_csv('../database/pseudo_observations/HYG.txt')
data3 = np.array(data3.iloc[::2, :])
data3 = np.array([float(i[0]) for i in data3])

data4 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data4 = np.array(data4.iloc[::2, :])
data4 = np.array([float(i[0]) for i in data4])

data5 = pd.read_csv('../database/pseudo_observations/IVV.txt')
data5 = np.array(data5.iloc[::2, :])
data5 = np.array([float(i[0]) for i in data5])

data6 = pd.read_csv('../database/pseudo_observations/GLD.txt')
data6 = np.array(data6.iloc[::2, :])
data6 = np.array([float(i[0]) for i in data6])

data7 = pd.read_csv('../database/pseudo_observations/1321.txt')
data7 = np.array(data7.iloc[::2, :])
data7 = np.array([float(i[0]) for i in data7])

data8 = pd.read_csv('../database/pseudo_observations/AUD.txt')
data8 = np.array(data8.iloc[::2, :])
data8 = np.array([float(i[0]) for i in data8])

data9 = pd.read_csv('../database/pseudo_observations/EUR.txt')
data9 = np.array(data9.iloc[::2, :])
data9 = np.array([float(i[0]) for i in data9])

data10 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data10 = np.array(data10.iloc[::2, :])
data10 = np.array([float(i[0]) for i in data10])

In [21]:
import pyvinecopulib as pv
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian,
                                             pv.BicopFamily.clayton,
                                             pv.BicopFamily.frank,
                                             pv.BicopFamily.gumbel,
                                             pv.BicopFamily.bb1])

u = np.stack([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10]).T
print(u)
cop = pv.Vinecop(u, controls=controls)

#u_sim = cop.simulate(500)
#print(u_sim)

[[0.71928319 0.27945011 0.47239207 ... 0.62038593 0.57277867 0.38955118]
 [0.4704488  0.52143145 0.50406523 ... 0.30084565 0.61411669 0.55198975]
 [0.71220833 0.60157051 0.5348449  ... 0.32420093 0.58407173 0.58271997]
 ...
 [0.65977126 0.43175131 0.33677661 ... 0.72261656 0.38388255 0.43092455]
 [0.52357397 0.37263047 0.53923004 ... 0.75875269 0.46857872 0.53012607]
 [0.20885459 0.47613649 0.61418096 ... 0.23369499 0.43436537 0.40347559]]


In [22]:
cop.structure

<pyvinecopulib.RVineStructure>
7 7 18 25 18 25 2 9 9 6 1 1 25 25 23 6 6 2 23 23 2 2 2 10 10 10 10 10 30 30 
17 25 8 5 2 1 12 19 6 16 11 25 23 23 2 16 2 10 3 2 12 12 10 30 30 30 30 30 10 
25 1 2 23 12 23 22 6 16 26 25 23 2 2 12 2 10 12 2 12 22 10 30 14 14 14 24 24 
1 23 12 2 22 2 10 16 26 2 23 2 12 12 3 10 12 22 12 22 10 30 14 20 20 24 14 
23 2 22 12 10 12 6 26 2 10 2 12 3 3 13 12 22 23 22 10 30 14 20 24 24 20 
2 12 10 3 6 3 16 2 10 12 12 3 5 13 22 22 23 3 10 30 14 20 24 4 4 
12 3 6 13 16 5 26 10 12 22 3 5 13 22 10 23 3 13 30 14 20 24 4 2 
3 5 16 22 26 11 9 12 22 23 5 13 22 10 6 3 13 30 14 20 24 4 12 
5 11 26 10 9 21 19 22 23 3 13 22 10 6 16 13 30 14 20 24 4 22 
11 21 9 1 19 13 29 23 3 13 22 10 6 16 26 30 14 20 24 4 23 
21 13 19 11 29 22 23 3 13 30 10 6 16 26 30 14 20 24 4 3 
13 22 29 21 23 10 3 13 30 25 6 16 26 30 14 20 24 4 13 
22 10 23 7 3 6 13 30 25 5 16 26 30 14 20 24 4 6 
10 15 3 6 13 16 30 25 5 1 26 30 14 20 24 4 16 
15 6 13 16 30 26 25 5 1 11 30 14 20 24 4 26 
6 16 30 26 25 30 5

In [4]:
# Simulate some data
np.random.seed(1234)  # seed for the random generator
n = 1000  # number of observations
d = 5  # the dimension
mean = np.random.normal(size=d)  # mean vector
cov = np.random.normal(size=(d, d))  # covariance matrix
cov = np.dot(cov.transpose(), cov)  # make it non-negative definite
x = np.random.multivariate_normal(mean, cov, n)

# Transform copula data using the empirical distribution
u = pv.to_pseudo_obs(x)
print(u)
# Fit a Gaussian vine
# (i.e., properly specified since the data is multivariate normal)
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])
cop = pv.Vinecop(u, controls=controls)

# Sample from the copula
n_sim = 1000
u_sim = cop.simulate(n_sim, seeds=[1, 2, 3, 4])

# Transform back simulations to the original scale
x_sim = np.asarray([np.quantile(x[:, i], u_sim[:, i]) for i in range(0, d)])

# Both the mean and covariance matrix look ok!
[mean, np.mean(x_sim, 1)]
[cov, np.cov(x_sim)]

[[0.62937063 0.75924076 0.22377622 0.23376623 0.34765235]
 [0.3976024  0.37862138 0.66133866 0.38361638 0.38761239]
 [0.31668332 0.67832168 0.52747253 0.74025974 0.46853147]
 ...
 [0.36563437 0.36563437 0.81518482 0.2997003  0.7042957 ]
 [0.47652348 0.53746254 0.11588412 0.51148851 0.37662338]
 [0.59340659 0.37962038 0.52147852 0.15984016 0.92007992]]


[array([[ 2.37095772,  1.72011592,  1.34581349, -2.33400987, -3.1412032 ],
        [ 1.72011592,  2.77391072, -0.8386675 , -1.94255384, -3.0356469 ],
        [ 1.34581349, -0.8386675 ,  4.73656299, -1.11520579, -1.00737741],
        [-2.33400987, -1.94255384, -1.11520579,  6.17099976, -0.86804342],
        [-3.1412032 , -3.0356469 , -1.00737741, -0.86804342, 10.39309954]]),
 array([[ 2.22422017,  1.5946634 ,  1.44467283, -2.24840034, -2.86432409],
        [ 1.5946634 ,  2.56490065, -0.64236225, -1.88381331, -2.55280137],
        [ 1.44467283, -0.64236225,  4.48378773, -1.21625326, -1.35825765],
        [-2.24840034, -1.88381331, -1.21625326,  6.11239488, -1.14967754],
        [-2.86432409, -2.55280137, -1.35825765, -1.14967754,  9.93512784]])]

In [18]:
from database.database_parser import create_connection
from database.database_parser import select_curr_solution
from distributions.student_poisson_mixture import spm

# Testing extracting optimal solutions
test_connection = create_connection('../database/abacus_database.db')
opt_sol = select_curr_solution(test_connection, 'HYG')

# Test running inverse CDF. ~ 5 seconds.
spm.ppf(0.5, 0, 1, 10, 0.15, 4)

      converged: True
           flag: 'converged'
 function_calls: 7
     iterations: 6
           root: 0.0004246556002506937